In [1]:
import flwr as fl
import numpy as np

class SimpleClient2(fl.client.NumPyClient):
    def get_parameters(self, config):
        params = [np.array([0.5, 0.6, 0.7])]
        print("Client 2: Sending parameters:", params)
        return params

    def fit(self, parameters, config):
        print("Client 2: Received parameters for training:", parameters)
        new_params = [p * 1.05 for p in parameters]  # Different update strategy
        print("Client 2: Sending updated parameters:", new_params)
        return new_params, len(parameters), {}

    def evaluate(self, parameters, config):
        print("Client 2: Evaluating model with parameters:", parameters)
        return 0.4, len(parameters), {}  # Different dummy loss

# Start Client 2
fl.client.start_client(
    server_address="127.0.0.1:8080",
    client=SimpleClient2().to_client(),
)


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: train message 22cf94cd-a348-410a-8783-dc41cd616c4c
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message e6a0604f-b4dc-4295-97ab-36b50461481a
INFO :      Sent reply
INFO :      
INFO :      Received: train message 70a9b0cf-af40-4688-87bb-d11581771fda
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message e332d284-1e09-43d0-8883-94429779c0b5
INFO :      Sent reply
INFO :      
INFO :      Received: train message 3389fadf-ce74-4db8-a7d8-6fde73fe576b
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message fa36de59-969d-48c8-9bce

Client 2: Received parameters for training: [array([0.1, 0.2, 0.3])]
Client 2: Sending updated parameters: [array([0.105, 0.21 , 0.315])]
Client 2: Evaluating model with parameters: [array([0.1075, 0.215 , 0.3225])]
Client 2: Received parameters for training: [array([0.1075, 0.215 , 0.3225])]
Client 2: Sending updated parameters: [array([0.112875, 0.22575 , 0.338625])]
Client 2: Evaluating model with parameters: [array([0.1155625, 0.231125 , 0.3466875])]
Client 2: Received parameters for training: [array([0.1155625, 0.231125 , 0.3466875])]
Client 2: Sending updated parameters: [array([0.12134063, 0.24268125, 0.36402188])]
Client 2: Evaluating model with parameters: [array([0.12422969, 0.24845938, 0.37268906])]


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import flwr as fl

# Define a simple PyTorch model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(2, 1)  # Dummy 2-input, 1-output model

    def forward(self, x):
        return self.fc(x)

# Create a local model
model = SimpleNet()

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Flower client
class FLClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        params = [p.detach().numpy() for p in model.parameters()]
        print("Client: Sending initial parameters")
        return params

    def set_parameters(self, parameters):
        for param_tensor, new_param in zip(model.parameters(), parameters):
            param_tensor.data = torch.tensor(new_param, dtype=torch.float32)

    def fit(self, parameters, config):
        print("Client: Training on local data...")

        # Set parameters received from the server
        self.set_parameters(parameters)

        # Dummy training loop
        X = torch.tensor([[0.1, 0.2], [0.3, 0.4]], dtype=torch.float32)
        y = torch.tensor([[0.5], [0.7]], dtype=torch.float32)

        for _ in range(5):  # Train for 5 epochs
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

        print(f"Client: Finished training, Loss = {loss.item()}")

        # Send updated model parameters
        updated_params = [p.detach().numpy() for p in model.parameters()]
        return updated_params, len(X), {}

    def evaluate(self, parameters, config):
        print("Client: Evaluating model...")
        self.set_parameters(parameters)
        loss = 0.3  # Dummy loss value
        return loss, 10, {}

fl.client.start_client(
    server_address="127.0.0.1:8080",
    client=FLClient().to_client(),  # Convert to new Flower client format
)



	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8080: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-14T05:21:59.8186049+00:00"}"
>

In [19]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        """Return model parameters (dummy values)."""
        return [np.random.rand(2, 2)]  # Returns a 2x2 matrix

    def fit(self, parameters, config):
        """Simulate training by modifying parameters."""
        print(f"📥 Received parameters: {parameters}")
        updated_params = [p + 0.01 for p in parameters]  # Add small update
        return updated_params, 10, {}  # Return updated params, training size, and empty metrics

    def evaluate(self, parameters, config):
        """Simulate evaluation with dummy loss and accuracy."""
        print("📊 Evaluating model...")
        return 0.5, 10, {"accuracy": 0.8}  # Dummy loss & accuracy

# Start the client
app = fl.client.ClientApp(lambda _: SimpleClient().to_client())


In [6]:
import flwr as fl
import numpy as np

class SimpleClient(fl.client.NumPyClient):
    def __init__(self):
        # Initialize model parameters (dummy model)
        self.weights = np.array([0.0, 0.0])

    def get_parameters(self, config):  # ✅ Added `config` argument
        """Return current model parameters."""
        return [self.weights]

    def fit(self, parameters, config):
        """Simulate local training."""
        print(f"📥 Training on received parameters: {parameters}")
        self.weights = parameters[0]  # Update local model
        self.weights += np.random.randn(*self.weights.shape) * 0.01  # Simulated update
        return [self.weights], len(self.weights), {}

    def evaluate(self, parameters, config):
        """Simulate evaluation."""
        print("📊 Evaluating model...")
        self.weights = parameters[0]  # Update local model
        loss = np.sum((self.weights - np.array([1.0, 1.0]))**2)  # Simulated loss
        return loss, len(self.weights), {"accuracy": 0.8}

# Start Flower client
fl.client.start_client(
    server_address="127.0.0.1:8080",
    client=SimpleClient().to_client(),
)


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message caefe1c1-2642-4ea0-8624-d8a31c677eb4
INFO :      Sent reply
INFO :      
INFO :      Received: train message 303351b5-69ca-43f2-87ab-f641af245df4
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message a513ecd1-1b97-4c12-92f3-ea753298714e
INFO :      Sent reply
INFO :      
INFO :      Received: train message 0041df32-1aaf-43b5-a215-dc4e1e6da54c
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message f3342178-9e74-40df-9649-ccfe4cbf7ae2
INFO :      Sent reply
INFO :      
INFO :      Received: train message 9a5bd54f-6055-480

📥 Training on received parameters: [array([0., 0.])]
📊 Evaluating model...
📥 Training on received parameters: [array([ 0.00606872, -0.01105825])]
📊 Evaluating model...
📥 Training on received parameters: [array([ 0.00341657, -0.00763055])]
📊 Evaluating model...
